# 🚀 Ailo Colab Miner (AILO-1B)

**Mine AiloCoin with Google Colab's free T4 GPU!**

1. Runtime > Change runtime type > T4 GPU
2. Enter wallet below
3. Runtime > Run all

✨ **NEW**: Now syncs with global model!

In [ ]:
#@title ⚙️ 1. Setup
!pip install torch --index-url https://download.pytorch.org/whl/cu118 -q
!pip install aiohttp requests -q
import gc; gc.collect()
import torch
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"✅ GPU: {torch.cuda.get_device_name(0)} | VRAM: {torch.cuda.get_device_properties(0).total_memory/1e9:.1f}GB")
else:
    print("❌ Enable GPU: Runtime > Change runtime type > T4")

In [ ]:
#@title 🔑 2. Wallet
WALLET = ""  #@param {type:"string"}
print(f"✅ {WALLET[:12]}..." if len(WALLET)>=40 else "❌ Enter wallet from https://ailo.site/wallet.html")

In [ ]:
#@title 🧠 3. AILO-1B Model + Global Sync
import torch, torch.nn as nn, numpy as np, requests, aiohttp, asyncio, time, base64, gzip, gc, os

API="https://ailo.site/api"
SERVER="https://ailo.site"
VER="1.5.0-colab"

class AILO1B(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb=nn.Embedding(50257,1600)
        self.pos=nn.Parameter(torch.zeros(1,512,1600))
        self.tf=nn.TransformerEncoder(nn.TransformerEncoderLayer(1600,25,6400,0.1,batch_first=True),24)
        self.out=nn.Linear(1600,50257)
    def forward(self,x):
        return self.out(self.tf(self.emb(x)*40+self.pos[:,:x.size(1)]))

class Trainer:
    def __init__(self):
        self.dev=torch.device('cuda')
        self.m=None; self.opt=None; self.steps=0; self.acc=0
        self.local_version=0
    
    def init(self):
        print("🧠 Loading AILO-1B...")
        gc.collect(); torch.cuda.empty_cache()
        self.m=AILO1B().half().to(self.dev)
        print(f"   {sum(p.numel() for p in self.m.parameters()):,} params | GPU: {torch.cuda.memory_allocated()/1e9:.1f}GB")
        self.opt=torch.optim.SGD(self.m.parameters(),lr=0.001,momentum=0.9)
        self.crit=nn.CrossEntropyLoss()
        print("   ✅ Ready!")
    
    def sync_with_global(self):
        """Sync with global aggregated gradients from server."""
        try:
            print("🔄 Checking for global model updates...")
            r=requests.get(f"{SERVER}/api/model/weights",timeout=30)
            if r.status_code!=200:
                print("   No global model yet, using local")
                return False
            
            data=r.json()
            srv_ver=data.get('version',0)
            
            if srv_ver<=self.local_version:
                print(f"   Already at v{self.local_version}")
                return True
            
            print(f"   Server has v{srv_ver}, local has v{self.local_version}")
            
            if data.get('downloadUrls',{}).get('gradients'):
                grad_url=f"{SERVER}{data['downloadUrls']['gradients']}"
                print(f"   📥 Downloading gradients v{srv_ver}...")
                gr=requests.get(grad_url,timeout=60)
                if gr.status_code==200:
                    self._apply_gradients(gr.content)
                    self.local_version=srv_ver
                    print(f"   ✅ Synced to v{srv_ver}")
                    return True
            return False
        except Exception as e:
            print(f"   ⚠️ Sync failed: {e}")
            return False
    
    def _apply_gradients(self, grad_bytes):
        """Apply aggregated gradients to output layer."""
        try:
            grads=np.frombuffer(grad_bytes,dtype=np.float16).astype(np.float32)
            print(f"   Received {len(grads):,} gradient values")
            with torch.no_grad():
                out_params=self.m.out.weight.numel()
                if len(grads)>=out_params:
                    lr=0.0001
                    gt=torch.from_numpy(grads[:out_params]).reshape_as(self.m.out.weight)
                    self.m.out.weight.data-=lr*gt.half().to(self.dev)
                    print(f"   Applied to output layer ({out_params:,} params)")
        except Exception as e:
            print(f"   ⚠️ Apply failed: {e}")
    
    def batch(self,w):
        try:
            r=requests.get(f"{API}/cuda/training-data",params={'batchSize':1,'wallet':w},timeout=10)
            if r.ok:
                t=r.json().get('articles',[''])[0]
                tk=[ord(c)%50257 for c in t[:33]]
                tk+=[0]*(33-len(tk))
                return torch.tensor([tk[:32]]),torch.tensor([tk[1:33]])
        except:pass
        return torch.randint(0,50257,(1,32)),torch.randint(0,50257,(1,32))
    
    def step(self,x,y):
        self.m.train()
        x,y=x.to(self.dev),y.to(self.dev)
        with torch.cuda.amp.autocast():
            loss=self.crit(self.m(x).view(-1,50257),y.view(-1))/16
        loss.backward()
        self.acc+=1
        if self.acc>=16:
            nn.utils.clip_grad_norm_(self.m.parameters(),1.0)
            self.opt.step(); self.opt.zero_grad(set_to_none=True); self.acc=0
        self.steps+=1
        if self.steps%50==0: gc.collect(); torch.cuda.empty_cache()
        return loss.item()*16
    
    def grads(self):
        g=self.m.out.weight.grad
        if g is None: return None
        gc.collect()
        flat=g.cpu().float().flatten()
        sample=flat[::10]
        del flat; gc.collect()
        comp=gzip.compress(sample.half().numpy().tobytes(),4)
        del sample; gc.collect()
        return base64.b64encode(comp).decode()

print("✅ Ready (with global sync)")

In [ ]:
#@title 🚀 4. Mine!

async def hb(w,tps):
    try:
        async with aiohttp.ClientSession() as s:
            await s.post(f"{API}/cuda/register",json={'wallet':w,'clientVersion':VER,'deviceInfo':{'gpu_name':'T4-Colab','vram_gb':15,'hashrate':tps}})
            await s.post(f"{API}/ping",json={'wallet':w,'status':'training','hashrate':f"{tps:.0f} Tok/s"})
    except:pass

async def submit(w,g,s,l):
    try:
        async with aiohttp.ClientSession() as ss:
            async with ss.post(f"{API}/cuda/submit",json={'wallet':w,'gradients':g,'epoch':s,'loss':l,'gpu':'T4-Colab'}) as r:
                if r.status==200: return (await r.json()).get('reward',0)
    except:pass
    return 0

async def run():
    if len(WALLET)<40: print("❌ Set wallet!"); return
    print("="*50)
    print(f"  AILO-1B Colab Miner v{VER}")
    print("  🔄 Now with Global Model Sync!")
    print("="*50)
    
    await hb(WALLET,0)
    t=Trainer(); t.init()
    
    # Initial sync with global model
    t.sync_with_global()
    
    print("\n⛏️ MINING!")
    print("📊 Dashboard: https://ailo.site/dashboard.html\n")
    
    rew,last_s,last_h,last_sync,tps=0.0,time.time(),time.time(),time.time(),0.0
    
    try:
        while True:
            t0=time.time()
            x,y=t.batch(WALLET)
            loss=t.step(x,y)
            tps=32/(time.time()-t0+0.001)
            
            if time.time()-last_h>=10: await hb(WALLET,tps); last_h=time.time()
            if t.steps%25==0: print(f"Step {t.steps} | Loss: {loss:.4f} | {tps:.0f} tok/s | v{t.local_version} | GPU: {torch.cuda.memory_allocated()/1e9:.1f}GB")
            
            if time.time()-last_s>=300:
                print("\n📤 Submitting gradients...")
                gc.collect(); torch.cuda.empty_cache()
                g=t.grads()
                if g:
                    r=await submit(WALLET,g,t.steps,loss); rew+=r
                    print(f"💰 +{r:.4f} ALC (Total: {rew:.4f})")
                last_s=time.time()
                
                # Sync every 10 minutes
                if time.time()-last_sync>=600:
                    t.sync_with_global()
                    last_sync=time.time()
                print()
                
    except KeyboardInterrupt: print(f"\n⏹️ Total: {rew:.4f} ALC")

await run()